In [ ]:
# 4차시 보강 실습: Bias-Variance 감각 익히기 (편ba 분va )
import torch, math, random
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

# 1) 데이터: 사인파 + 잡음 -> 회귀 문제
torch.manual_seed(0)
N = 600
x = torch.linspace(-3*math.pi, 3*math.pi, N).unsqueeze(1)
y = torch.sin(x) + 0.2*torch.randn_like(x)

X_train, X_temp, y_train, y_temp = train_test_split(x.numpy(), y.numpy(), test_size=0.4, random_state=42)
X_val,   X_test, y_val,   y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

X_train, y_train = torch.tensor(X_train, dtype=torch.float32), torch.tensor(y_train, dtype=torch.float32)
X_val,   y_val   = torch.tensor(X_val,   dtype=torch.float32), torch.tensor(y_val,   dtype=torch.float32)
X_test,  y_test  = torch.tensor(X_test,  dtype=torch.float32), torch.tensor(y_test,  dtype=torch.float32)

# 2) 두 모델: 저용량(작은 MLP) vs 고용량(큰 MLP)
def make_mlp(hidden):
    return nn.Sequential(
        nn.Linear(1, hidden), nn.ReLU(),
        nn.Linear(hidden, hidden), nn.ReLU(),
        nn.Linear(hidden, 1)
    )

small = make_mlp(hidden=8)   # 고편향/저분산 경향
big   = make_mlp(hidden=128) # 저편향/고분산 경향

def train(model, Xtr, ytr, Xva, yva, epochs=600, lr=1e-3):
    # lr = [0.001,0.01,0.1] for i in lr 가장 성능 좋게 나온 lr >> lr = lr
    opt = optim.Adam(model.parameters(), lr=lr)
    loss_fn = nn.MSELoss()
    tr_hist, va_hist = [], []
    for ep in range(epochs):
        model.train()
        opt.zero_grad()
        pred = model(Xtr) # 반드시 훈련용 데이터로 학습(**) >> 예측 값
        loss = loss_fn(pred, ytr)
        loss.backward()   # w1 = w0 - lr(diff_y / diff_xtr)
        opt.step()

        # transfer learning(전이 학습)
        # 잘 된 모델(오픈소스 제공 모델)을 활용, 내 모델에 적용하는 것
        model.eval()  # eval : evaluation 평가
        with torch.no_grad():
            v = loss_fn(model(Xva), yva).item()
        tr_hist.append(loss.item()); va_hist.append(v)
    return tr_hist, va_hist

tr_s, va_s = train(small, X_train, y_train, X_val, y_val, epochs=400)
tr_b, va_b = train(big, X_train, y_train, X_val, y_val, epochs=400)

plt.figure(); plt.plot(tr_s, label='small-train'); plt.plot(va_s, label='small-val')
plt.plot(tr_b, label='big-train'); plt.plot(va_b, label='big-val'); plt.legend(); plt.title('Bias-Variance')
plt.show()

# 최종 테스트 MSE
def mse(model, X, y):
    with torch.no_grad():
        return nn.MSELoss()(model(X), y).item()
print("Small Test MSE:", mse(small, X_test, y_test))
print("Big   Test MSE:", mse(big,   X_test, y_test))
